# Bidding

This is a tutorial for how to use the bidding engine

In [1]:
import sys
import os
sys.path.append('../../src')
os.environ['BEN_HOME'] = "../.."

from nn.models import Models
from bots import BotBid
from util import hand_to_str
from sample import Sample
import conf
import numpy as np

np.set_printoptions(precision=2, suppress=True, linewidth=200)
np.random.seed(42)


In [2]:
models = Models.from_conf(conf.load('../Conf/UCBC2024.conf'),'..')   # loading neural networks
sampler = Sample.from_conf(conf.load('../Conf/UCBC2024.conf'), False)  # Load sampling strategies



Instructions for updating:
non-resource variables are not supported in the long term


INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-bidding_V2-3125000
INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-bidding_V2-3125000
INFO:tensorflow:Restoring parameters from ..\Models/contract-193200
INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-binfo_V2-3125000
INFO:tensorflow:Restoring parameters from ..\Models/lead_suit-999000
INFO:tensorflow:Restoring parameters from ..\Models/lead_nt-475000
INFO:tensorflow:Restoring parameters from ..\Models/Jack/lr3-1000000
INFO:tensorflow:Restoring parameters from ..\Models/single_dummy-32768000
INFO:tensorflow:Restoring parameters from ..\Models/lefty_nt-475000
INFO:tensorflow:Restoring parameters from ..\Models/dummy_nt-475000
INFO:tensorflow:Restoring parameters from ..\Models/righty_nt-475000
INFO:tensorflow:Restoring parameters from ..\Models/decl_nt-475000
INFO:tensorflow:Restoring parameters from ..\Models/lefty_

### Running through an example

In [3]:
# East deals, EW vulnerable.
vuln_ns, vuln_ew = False, False

from ddsolver import ddsolver
dds = ddsolver.DDSolver()

# you sit West and hold:
hand = 'K62.Q7542.J832.7'

auction = ["PASS", "2N", "PASS", "3D","PASS", "4C", "PASS"]
bot_bid = BotBid([vuln_ns, vuln_ew], hand, models, sampler, 3, 0, dds, False)

bid = bot_bid.bid(auction)
bid.to_dict()['candidates']

# what's your bid?

Loaded lib dds.dll
DDSolver being loaded - dds mode 1


[{'call': '4H',
  'insta_score': 0.518,
  'adjustment': 259,
  'expected_score': 432,
  'expected_tricks': 11.2},
 {'call': '4D',
  'insta_score': 0.279,
  'adjustment': 139,
  'expected_score': 386,
  'expected_tricks': 11.2},
 {'call': '4N',
  'insta_score': 0.196,
  'adjustment': 98,
  'expected_score': 307,
  'expected_tricks': 11.2}]

in the output above:
- `insta_score` reflects what the neural network prefers to bid
- `expected_score` is based on what you expect to score on difference samples (for example, 70% of the time you are making and 30% of the time you are going one off, then the expected score is 0.7 * 620 - 0.3 * 100)

### Samples consistent with the auction so far

Above we said that we are computig expected scores on samples consistent with the auction so far.

How do we get these samples?

We get some samples in the response we get from the bot (i.e `bid.samples`)

In [4]:
bid.samples

['Q.JTx.Q9x.J98xxx xxxx.AK98.AKx.AK AJT98.x.Txx.QTxx Kxx.Qxxxx.J8xx.x 0.55615 \n P-2N-P-3D-P-4C-P-4H-P-P-P (10, 10.1)  \n P-2N-P-3D-P-4C-P-4D-P-4N-P-5C-P-6H-P-P-P (10, 10.2)  \n P-2N-P-3D-P-4C-P-4N-P-5D-P-5H-P-P-P (10, 10.1) ',
 'T98x.T8x.T.AQ98x AJxx.AKJ9.Axx.KJ Qx.x.KQ9xx.Txxxx Kxx.Qxxxx.J8xx.x 0.54980 \n P-2N-P-3D-P-4C-P-4H-P-P-P (11, 10.2)  \n P-2N-P-3D-P-4C-P-4D-P-4H-P-P-P (11, 10.1)  \n P-2N-P-3D-P-4C-P-4N-P-5D-P-5H-P-P-P (11, 10.2) ',
 'Jxxx.T.Axx.KJTxx A9x.AK8x.KT9x.AQ QT8.J9x.Qx.98xxx Kxx.Qxxxx.J8xx.x 0.54199 \n P-2N-P-3D-P-4C-P-4H-P-P-P (11, 10.9)  \n P-2N-P-3D-P-4C-P-4D-P-4H-P-P-P (11, 11.1)  \n P-2N-P-3D-P-4C-P-4N-P-5C-P-5H-P-P-P (11, 10.9) ',
 'T9x.Tx.QT.JT9xxx xx.AKJx.A9xx.AKQ AQJ8x.98.Kxx.8xx Kxx.Qxxxx.J8xx.x 0.53906 \n P-2N-P-3D-P-4C-P-4H-P-P-P (11, 10.4)  \n P-2N-P-3D-P-4C-P-4D-P-4N-P-5C-P-6H-P-P-P (11, 10.7)  \n P-2N-P-3D-P-4C-P-4N-P-5C-P-5H-P-P-P (11, 10.4) ',
 'Q9xx.T8.Tx.AJxxx Axx.AKJx.AQxx.KT JT8.9x.K9x.Q98xx Kxx.Qxxxx.J8xx.x 0.53662 \n P-2N-P-3D-P-4C-P-4H-P-P-P (

each row is one sample. the hands are in the order: N E S W

(West always has the same cards because it's your hand and it's known to you)

In [5]:
# we got some samples above. if we want more, we can always get more

sample_hands, sorted_score, p_hcp, p_shp, good_quality = bot_bid.sample_hands_for_auction(["PAD_START", "PAD_START", "PAD_START", "PASS", "1N", "PASS", "3S","PASS"],0)
print("Good quality:", good_quality)
for i in range(sample_hands.shape[0]):
    print(f'{hand_to_str(sample_hands[i, 0, :])} {hand_to_str(sample_hands[i, 1, :])} {hand_to_str(sample_hands[i, 2, :])} {hand_to_str(sample_hands[i, 3, :])}')

Good quality: 0.824
Kxx.Qxxxx.J8xx.x AJT98.9x.ATx.JTx x.AT8x.Qxx.AKxxx Qxxx.KJ.K9x.Q98x
Kxx.Qxxxx.J8xx.x AT9xx.x.Q9xx.Kxx x.AJ9x.KTx.AQJ8x QJ8x.KT8.Ax.T9xx
Kxx.Qxxxx.J8xx.x ATxxx.J8.Qx.QT8x J.KT9x.K9x.AKJxx Q98x.Ax.ATxx.9xx
Kxx.Qxxxx.J8xx.x AQ98x.9.Txx.A8xx x.AJ8x.A9x.KQTxx JTxx.KTx.KQx.J9x
Kxx.Qxxxx.J8xx.x AT9xx.T9.Q9xx.A9 8.AJ8x.AKT.QJTxx QJxx.Kx.xx.K8xxx
Kxx.Qxxxx.J8xx.x AQxx.K8.Kxxx.K9x 8.AT9x.A9x.AQJxx JT9xx.Jx.QT.T8xx
Kxx.Qxxxx.J8xx.x A8xxx.9.Qx.A8xxx x.AKxx.Kxxx.KT9x QJT9.JT8.AT9.QJx
Kxx.Qxxxx.J8xx.x AJ98xx.Tx.A.98xx x.KJ9x.KQ9x.AKJx QTx.A8.Txxx.QTxx
Kxx.Qxxxx.J8xx.x AQJT8x.x.T9.KJTx .AKTx.KQxx.AQ8xx 9xxx.J98.Axx.9xx
Kxx.Qxxxx.J8xx.x QJTxx.x.AK9x.Txx 8.AKJ9.Qxxx.AQ9x A9xx.T8x.T.KJ8xx
Kxx.Qxxxx.J8xx.x AQJ98x.Jx.T.Kxxx x.KT98.AK9x.AQ8x Txx.Ax.Qxxx.JT9x
Kxx.Qxxxx.J8xx.x A8xx.T8x.KQ9x.Q8 J.AKJ9.Ax.KTxxxx QT9xx.x.Txx.AJ9x
Kxx.Qxxxx.J8xx.x AT8xx.J9.x.QJ9xx J.AKxx.Qxxx.AKxx Q9xx.T8.AKT9.T8x
Kxx.Qxxxx.J8xx.x AQTx.K9x.Qxx.T9x 9.AJTx.A9x.AKQxx J8xxx.8.KTx.J8xx
Kxx.Qxxxx.J8xx.x ATxx.KJ8.xx